In [18]:
import numpy as np
from skimage import feature, filters
import pickle
import cv2
from skimage.feature import hog

In [19]:
# Load test data
Xts = np.loadtxt("/content/drive/MyDrive/TestData.csv")

# Reshape test data to 28x28
Xts = Xts.reshape(-1, 28, 28)

In [20]:
def extract_features(images):
    features = []
    
    for image in images:
        # Resize the image to at least 16x16 pixels
        resized_image = cv2.resize(image, (16, 16))

        # Normalize the pixel values to 0-1
        normalized_image = resized_image / 255.0

        # Compute the HoG features
        hog_features = hog(normalized_image, orientations=9, pixels_per_cell=(8, 8),
                           cells_per_block=(2, 2), visualize=False, feature_vector=True)
        
        # Perform Canny Edge Detection on the normalized image
        edges = cv2.Canny((normalized_image * 255).astype('uint8'), 100, 200)

        # Flatten the edges image to a 1D array
        edges_flat = edges.flatten()

        # Combine the HoG features and Canny edges into a single feature vector
        combined_features = np.concatenate((hog_features, edges_flat))

        features.append(combined_features)
    
    return features

In [21]:
def load_model(filename):
    # Load the saved model
    with open(filename, "rb") as file:
        model = pickle.load(file)

    return model

In [22]:
def save_predictions(predictions, filename):
    # Save the predictions to a CSV file
    np.savetxt(filename, predictions)

In [23]:
if __name__ == '__main__':
    # Extract features from test data
    x_test_features = extract_features(Xts)

    # Load the saved model
    model = load_model("/content/drive/MyDrive/myModel.pkl")

    # Generate predictions for the test data
    predictions = model.predict(x_test_features)

    # Save the predictions to a CSV file
    save_predictions(predictions, "myPredictions.csv")